# Aula 7 - Segmentação de Dados

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

### Exemplo Scikit-Learn

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape(-1, 1)
y = np.array([3, 5, 7, 9, 11, 13, 15, 17, 19, 21]).reshape(-1, 1)

sns.scatterplot(x=x.flatten(), y=y.flatten())

In [ ]:
linear = LinearRegression()
linear.fit(x, y)

m = linear.coef_[0][0]
b = linear.intercept_[0]

print(f"Fórmula: y = {m}x + {b}")

plt.plot(x, linear.predict(x), color='red')
plt.plot(x.flatten(), y.flatten(), 'o', color='blue')
plt.text(6, 10, f"y = {m}x + {b}", fontsize=12)

print(f"Se x = 100, y = {linear.predict([[100]])}")

### Segmentação

In [ ]:
df = pd.read_excel('../bases/clientes.xlsx')
df

In [ ]:
df['Gênero'] = df['Gênero'].map({'Feminino': 0, 'Masculino': 1})
df['Educação'] = df['Educação'].map({'Ensino Fundamental': 1, 'Ensino Médio': 2, 'Ensino Superior': 3, 'Pós-Graduação': 4})

X = df.values

In [ ]:
X = df.values
scaler = StandardScaler()
X = scaler.fit_transform(X)
pca = PCA(n_components=2)
X = pca.fit_transform(X)

plt.figure(figsize=(12, 6))
sns.scatterplot(x=X[:, 0], y=X[:, 1])
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x=['PC1', 'PC2'], y=pca.explained_variance_ratio_)
plt.title('Variância explicada por cada componente (Scree Plot)')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4)

kmeans.fit(X)

df['Cluster'] = kmeans.labels_

plt.figure(figsize=(12, 6))
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=df['Cluster'], palette='viridis')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5)

kmeans.fit(X)

df['Cluster'] = kmeans.labels_

plt.figure(figsize=(12, 6))
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=df['Cluster'], palette='viridis')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=6)

kmeans.fit(X)

df['Cluster'] = kmeans.labels_

plt.figure(figsize=(12, 6))
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=df['Cluster'], palette='viridis')
plt.show()

In [ ]:
inercia = []
for i in range(2, 15):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(X)
    df['Cluster'] = kmeans.labels_
    inercia.append(kmeans.inertia_)

plt.figure(figsize=(12, 6))
plt.plot(range(2, 15), inercia, marker='o')
# plt.plot(3, inercia[1], marker='o', markersize=15, markeredgewidth=2, markerfacecolor='red', markeredgecolor='red')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.show()

In [ ]:
diff = np.diff(inercia)
maior_dif = np.argmin(diff) + 1
print(f'O número ideal de clusters é {list(range(2, 15))[maior_dif]}')

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=93)

kmeans.fit(X)

df['Cluster'] = kmeans.labels_

plt.figure(figsize=(12, 6))
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=df['Cluster'], palette='viridis')
plt.show()

In [ ]:
df.groupby('Cluster').agg({'Idade': ['min', 'max', 'median'], 'Renda_Mensal': 'mean', 'Compras_Anuais': 'mean', 'Educação': ['min', 'max', 'median']})

In [ ]:
df['Gênero'] = df['Gênero'].map({0: 'Feminino', 1: 'Masculino'})
df['Educação'] = df['Educação'].map({1: 'Ensino Fundamental', 2: 'Ensino Médio', 3: 'Ensino Superior', 4: 'Pós-Graduação'})

fig, ax = plt.subplots(nrows=3, ncols=5, figsize=(20, 15))

for cluster in range(3): # Temos 3 clusters! 
    sns.countplot(data=df[df['Cluster'] == cluster], x='Gênero', ax=ax[cluster, 0], palette='viridis')
    sns.histplot(data=df[df['Cluster'] == cluster], x='Idade', ax=ax[cluster, 1], kde=True, palette='viridis')
    ax[cluster, 1].set_xlim(18, 70)
    sns.histplot(data=df[df['Cluster'] == cluster], x='Compras_Anuais', ax=ax[cluster, 2], kde=True, palette='viridis', bins=5)
    ax[cluster, 2].set_xlim(5, 50)
    sns.histplot(data=df[df['Cluster'] == cluster], x='Renda_Mensal', ax=ax[cluster, 3], kde=True, palette='viridis')
    ax[cluster, 3].set_xlim(1000, 20000)
    ax[cluster, 0].set_title(f'Cluster {cluster}')
    sns.countplot(data=df[df['Cluster'] == cluster], x='Educação', ax=ax[cluster, 4], palette='viridis')
    ax[cluster, 4].set_xticklabels(['EF', 'EM', 'ES', 'Pós'])
    ax[cluster, 1].set_ylabel('')
    ax[cluster, 2].set_ylabel('')
    ax[cluster, 3].set_ylabel('')
    ax[cluster, 4].set_ylabel('')